In [1]:
import os
import re
import pandas as pd
from tqdm import tqdm

### Paths

In [151]:
train_files_directory = "/Utilisateurs/umushtaq/am_reasoning/abstRCT/raw_files/train/neoplasm_train"
dev_files_directory = "/Utilisateurs/umushtaq/am_reasoning/abstRCT/raw_files/dev/neoplasm_dev"
neo_test_files_directory = "/Utilisateurs/umushtaq/am_reasoning/abstRCT/raw_files/test/neoplasm_test"
mix_test_files_directory = "/Utilisateurs/umushtaq/am_reasoning/abstRCT/raw_files/test/mixed_test"
gla_test_files_directory = "/Utilisateurs/umushtaq/am_reasoning/abstRCT/raw_files/test/glaucoma_test"

### Process ANN files

In [152]:
# def process_ann_files(dir):
    
#     component_data = []
#     relations_data = []
    
#     for filename in tqdm([f for f in os.listdir(dir) if f.endswith('.ann')], desc="Parsing annotation files for components ..."):
    
#         #print("processing: " + filename)
#         #if filename.endswith(".ann"):  # Process only text files
#         file_path = os.path.join(dir, filename)
        
#         with open(file_path, "r", encoding="utf-8") as file:
#             for line in file:
#                 parts = line.strip().split("\t")
#                 #print(parts)
#                 # Process "T" lines (claims/premises)
#                 if line.startswith("T"):
#                     t_id = parts[0]  # Extract T ID (e.g., T1)
#                     t_type = parts[1].split(" ")[0]  # Extract type (e.g., MajorClaim, Claim, Premise)
#                     t_s_bound = parts[1].split(" ")[1]
#                     t_e_bound = parts[1].split(" ")[2]
#                     text = parts[2] #if len(parts) > 3 else ""  # Extract text if available
#                     component_data.append(["T", filename, t_id, t_type, t_s_bound, t_e_bound, text])
                    
#     for filename in tqdm([f for f in os.listdir(dir) if f.endswith('.ann')], desc="Parsing annotation files for relations ... "):
    
#         #print("processing: " + filename)
#         #if filename.endswith(".ann"):  # Process only text files
#         file_path = os.path.join(dir, filename)
        
#         with open(file_path, "r", encoding="utf-8") as file:
#             has_relation = False
#             for line in file:
#                 parts = line.strip().split("\t")
#                 #print(parts)
#                 # Process "T" lines (claims/premises)
#                 if line.startswith("R"):
#                     has_relation = True
#                     parts = line.strip().split()
#                     relation_type = parts[1]  # "supports" or "attacks"
#                     arg1 = parts[2].split(":")[1]  # Extract T value from Arg1
#                     arg2 = parts[3].split(":")[1]  # Extract T value from Arg2
#                     relations_data.append(["R", filename, arg1, arg2, relation_type])
                    
#             if not has_relation:
#                 relations_data.append(["R", filename, None, None, None])
                    
#     return component_data, relations_data

In [153]:
def process_ann_files(dir):
    
    component_data = []
    relations_data = []
    
    for filename in tqdm([f for f in os.listdir(dir) if f.endswith('.ann')], desc="Parsing annotation files ..."):
    
        file_path = os.path.join(dir, filename)
        
        with open(file_path, "r", encoding="utf-8") as file:
            has_relation = False
            for line in file:
                parts = line.strip().split("\t")
                #print(parts)
                # Process "T" lines (claims/premises)
                if line.startswith("T"):
                    t_id = parts[0]  # Extract T ID (e.g., T1)
                    t_type = parts[1].split(" ")[0]  # Extract type (e.g., MajorClaim, Claim, Premise)
                    t_s_bound = parts[1].split(" ")[1]
                    t_e_bound = parts[1].split(" ")[2]
                    text = parts[2] #if len(parts) > 3 else ""  # Extract text if available
                    component_data.append(["T", filename, t_id, t_type, t_s_bound, t_e_bound, text])                    
                    
                elif line.startswith("R"):
                    has_relation = True
                    parts = line.strip().split()
                    relation_type = parts[1]  # "supports" or "attacks"
                    arg1 = parts[2].split(":")[1]  # Extract T value from Arg1
                    arg2 = parts[3].split(":")[1]  # Extract T value from Arg2
                    relations_data.append(["R", filename, arg1, arg2, relation_type])
                    
            if not has_relation:
                relations_data.append(["R", filename, None, None, None])
                    
    return component_data, relations_data

In [154]:
train_component_data, train_relations_data = process_ann_files(train_files_directory)

Parsing annotation files ...: 100%|██████████| 350/350 [00:00<00:00, 1335.57it/s]


In [155]:
dev_component_data, dev_relations_data = process_ann_files(dev_files_directory)

Parsing annotation files ...: 100%|██████████| 50/50 [00:00<00:00, 1401.89it/s]

In [156]:
neo_test_component_data, neo_test_relations_data = process_ann_files(neo_test_files_directory)

Parsing annotation files ...:   0%|          | 0/100 [00:00<?, ?it/s]

Parsing annotation files ...: 100%|██████████| 100/100 [00:00<00:00, 1438.32it/s]


In [157]:
mix_test_component_data, mix_test_relations_data = process_ann_files(mix_test_files_directory)

Parsing annotation files ...:   0%|          | 0/100 [00:00<?, ?it/s]

Parsing annotation files ...: 100%|██████████| 100/100 [00:00<00:00, 1328.11it/s]


In [158]:
gla_test_component_data, gla_test_relations_data = process_ann_files(gla_test_files_directory)

Parsing annotation files ...:   0%|          | 0/100 [00:00<?, ?it/s]

Parsing annotation files ...: 100%|██████████| 100/100 [00:00<00:00, 1393.77it/s]


### Build DataFrames

In [159]:
def build_dataframes(components, relations):
    
    comp_df = pd.DataFrame(components)
    comp_df.columns = ["type_indicator", "filename", "ac_id", "ac_type", "ac_start_bound", "ac_end_bound", "ac"]
    
    r_df = pd.DataFrame(relations)
    r_df.columns = ["type_indicator", "filename", "source", "target", "relation_type"]
    
    train_gdf = comp_df.groupby(["filename"], sort=False).agg({
    'type_indicator': list,
    'ac_id': list,
    "ac_type": list,
    'ac_start_bound': list,
    'ac_end_bound': list,
    'ac': list,
}).reset_index()
    
    rtrain_gdf = r_df.groupby(["filename"], sort=False).agg({
    'type_indicator': list,
    'source': list,
    "target": list,
    'relation_type': list,
}).reset_index()
    
    return train_gdf.merge(rtrain_gdf, on="filename")   
    

In [160]:
train_df = build_dataframes(train_component_data, train_relations_data)

In [161]:
dev_df = build_dataframes(dev_component_data, dev_relations_data)

In [162]:
gla_test_df = build_dataframes(gla_test_component_data, gla_test_relations_data)
mix_test_df = build_dataframes(mix_test_component_data, mix_test_relations_data)
neo_test_df = build_dataframes(neo_test_component_data, neo_test_relations_data)

In [163]:
train_df.shape, dev_df.shape, gla_test_df.shape, mix_test_df.shape, neo_test_df.shape

((350, 11), (50, 11), (100, 11), (100, 11), (100, 11))

### Build Text Files

In [164]:
def process_text_files(dir):
    text_data = []

    for filename in tqdm([f for f in os.listdir(dir) if f.endswith('.txt')], desc="Parsing text files ..."):
        file_path = os.path.join(dir, filename)

        with open(file_path, "r", encoding="utf-8") as file:
            full_text = file.read()
            text_data.append([filename, filename.replace(".txt", ".ann"), full_text])

    return text_data

In [165]:
train_texts = process_text_files(train_files_directory)

Parsing text files ...:   0%|          | 0/350 [00:00<?, ?it/s]

Parsing text files ...: 100%|██████████| 350/350 [00:00<00:00, 1854.75it/s]


In [166]:
dev_texts = process_text_files(dev_files_directory)

Parsing text files ...:   0%|          | 0/50 [00:00<?, ?it/s]

Parsing text files ...: 100%|██████████| 50/50 [00:00<00:00, 1703.16it/s]


In [167]:
gla_test_texts = process_text_files(gla_test_files_directory)
mix_test_texts = process_text_files(mix_test_files_directory)
neo_test_texts = process_text_files(neo_test_files_directory)

Parsing text files ...:   0%|          | 0/100 [00:00<?, ?it/s]

Parsing text files ...: 100%|██████████| 100/100 [00:00<00:00, 1435.61it/s]


In [168]:
len(train_texts), len(dev_texts), len(gla_test_texts), len(mix_test_texts), len(neo_test_texts)

(350, 50, 100, 100, 100)

### Merge with dfs

In [169]:
train_texts_df = pd.DataFrame(train_texts)
dev_texts_df = pd.DataFrame(dev_texts)
gla_test_texts_df = pd.DataFrame(gla_test_texts)
mix_test_texts_df = pd.DataFrame(mix_test_texts)
neo_test_texts_df = pd.DataFrame(neo_test_texts)

In [170]:
train_texts_df.columns = ["text_file", "filename", "abstract_text"]
dev_texts_df.columns = ["text_file", "filename", "abstract_text"]
gla_test_texts_df.columns = ["text_file", "filename", "abstract_text"]
mix_test_texts_df.columns = ["text_file", "filename", "abstract_text"]
neo_test_texts_df.columns = ["text_file", "filename", "abstract_text"]

In [171]:
train_df_abs = train_df.merge(train_texts_df, on="filename")

In [172]:
dev_df_abs = dev_df.merge(dev_texts_df, on="filename")

In [173]:
gla_test_df_abs = gla_test_df.merge(gla_test_texts_df, on="filename")
mix_test_df_abs = mix_test_df.merge(mix_test_texts_df, on="filename")
neo_test_df_abs = neo_test_df.merge(neo_test_texts_df, on="filename")

In [174]:
train_df_abs.shape, dev_df_abs.shape, gla_test_df_abs.shape, mix_test_df_abs.shape, neo_test_df_abs.shape

((350, 13), (50, 13), (100, 13), (100, 13), (100, 13))

### Save DataFrames

In [ ]:
train_df_abs.shape, dev_df_abs.shape, gla_test_df_abs.shape, mix_test_df_abs.shape, neo_test_df_abs.shape

In [176]:
train_df_abs


,filename,type_indicator_x,ac_id,ac_type,ac_start_bound,ac_end_bound,ac,type_indicator_y,source,target,relation_type,text_file,abstract_text
0,10561201.ann,"[T, T, T, T, T, T, T]","[T1, T2, T3, T4, T6, T7, T8]","[MajorClaim, Premise, Premise, Premise, Premis...","[1, 992, 1063, 1188, 1452, 1569, 1769]","[162, 1058, 1187, 1447, 1568, 1768, 1945]",[A combination of mitoxantrone plus prednisone...,"[R, R, R, R, R, R]","[T8, T2, T3, T6, T4, T7]","[T1, T8, T8, T8, T8, T1]","[Support, Support, Support, Support, Support, ...",10561201.txt,A combination of mitoxantrone plus prednisone...
1,10561203.ann,"[T, T, T, T, T, T, T]","[T1, T3, T4, T5, T6, T7, T8]","[Claim, Premise, Premise, Premise, Premise, Pr...","[1, 1354, 1443, 1593, 1828, 1979, 2056]","[318, 1442, 1592, 1827, 1978, 2055, 2167]",[In endocrine therapy trials in advanced breas...,"[R, R, R, R, R]","[T7, T3, T4, T5, T6]","[T8, T8, T8, T8, T8]","[Partial-Attack, Support, Support, Support, Pa...",10561203.txt,In endocrine therapy trials in advanced breas...
2,10653877.ann,"[T, T, T, T, T, T, T, T, T]","[T1, T2, T3, T4, T5, T6, T7, T8, T9]","[Claim, Premise, Premise, Premise, Premise, Pr...","[1, 957, 1204, 1297, 1465, 1612, 1865, 1938, 2...","[156, 1203, 1296, 1464, 1611, 1864, 1928, 2001...",[Treatment with cisplatin-based chemotherapy p...,"[R, R, R, R, R]","[T8, T6, T2, T4, T5]","[T9, T9, T9, T9, T9]","[Support, Support, Support, Support, Support]",10653877.txt,Treatment with cisplatin-based chemotherapy p...
3,10675381.ann,"[T, T, T, T, T, T, T, T]","[T1, T2, T3, T4, T5, T6, T7, T8]","[MajorClaim, Premise, Premise, Premise, Premis...","[218, 1028, 1209, 1395, 1580, 1647, 1844, 2193]","[449, 1208, 1394, 1579, 1646, 1843, 2152, 2288]",[In nonrandomized studies involving patients w...,"[R, R, R, R, R]","[T2, T4, T5, T6, T7]","[T8, T8, T8, T8, T8]","[Support, Support, Support, Support, Support]",10675381.txt,Extracellular adenosine 5'-triphosphate (ATP)...
4,10735887.ann,"[T, T, T, T, T, T, T]","[T1, T2, T3, T4, T5, T6, T7]","[Premise, Premise, Premise, Premise, Claim, Pr...","[552, 678, 758, 1102, 1196, 1228, 1306]","[677, 757, 1101, 1195, 1227, 1305, 1555]",[Overall objective response (OR) rates were hi...,"[R, R, R, R]","[T1, T3, T4, T5]","[T7, T7, T7, T7]","[Support, Support, Support, Support]",10735887.txt,"This phase III, double-blind, randomized, mul..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,9807987.ann,"[T, T, T, T, T]","[T1, T2, T3, T4, T5]","[Premise, Premise, Premise, Premise, Claim]","[812, 1026, 1140, 1342, 1469]","[1025, 1139, 1341, 1468, 1760]","[With a median follow-up of 13 months, the ove...","[R, R, R]","[T4, T3, T1]","[T5, T5, T5]","[Support, Support, Support]",9807987.txt,"In phase II studies, irinotecan is active in ..."
346,9849452.ann,"[T, T, T, T, T, T, T, T, T]","[T1, T2, T3, T4, T5, T6, T7, T8, T9]","[Premise, Premise, Premise, Premise, Premise, ...","[983, 1124, 1187, 1250, 1327, 1349, 1503, 1624...","[1123, 1186, 1249, 1326, 1348, 1502, 1623, 166...",[Patients in the chemotherapy group reported b...,"[R, R, R, R, R]","[T5, T1, T3, T6, T7]","[T4, T9, T2, T9, T9]","[Partial-Attack, Support, Partial-Attack, Supp...",9849452.txt,The aim of the present trial was to evaluate ...
347,9849454.ann,"[T, T, T, T, T, T]","[T1, T2, T3, T4, T5, T6]","[Premise, Premise, Premise, Premise, Claim, Cl...","[1149, 1215, 1381, 1507, 2258, 2523]","[1214, 1380, 1506, 1699, 2522, 2655]",[Complete remission was achieved in 91% (170/1...,"[R, R, R, R]","[T1, T4, T3, T2]","[T6, T6, T5, T5]","[Support, Support, Support, Support]",9849454.txt,The second International Society of Paediatri...
348,9850014.ann,"[T, T, T, T, T, T]","[T1, T2, T3, T4, T5, T6]","[Premise, Premise, Premise, Premise, Premise, ...","[891, 1041, 1139, 1264, 1357, 1578]","[1040, 1138, 1263, 1356, 1577, 1770]",[An objective response (complete [CR] or parti...,"[R, R, R]","[T4, T5, T3]","[T6, T6, T6]","[Support, Support, Support]",9850014.txt,We report results of a randomized prospectiv